In [28]:
import pandas as pd
from simpletransformers.classification import ClassificationModel
import sklearn
from sklearn.model_selection import train_test_split

# read the input files
df = pd.read_json("classification_train.jsonl", lines=True)
test = pd.read_json("classification_test.jsonl", lines=True)

In [29]:
# translation -> English ['Sport', 'World', 'Culture', 'Technology', 'Local']
labels = ['Sport', 'Dunyo', 'Madaniyat', 'Texnologiyalar', 'Mahalliy']

In [30]:
for index, row in df.iterrows():
    df['category'][index] = int(labels.index(df['category'][index]))
    
for index, row in test.iterrows():
    test['category'][index] = int(labels.index(test['category'][index]))

<ipython-input-30-0cf17a7e1dab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'][index] = int(labels.index(df['category'][index]))
<ipython-input-30-0cf17a7e1dab>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['category'][index] = int(labels.index(test['category'][index]))


In [31]:
test

,content,category,title,date,url
0,“Odnoklassniki” ijtimoiy tarmog‘i Smart TV uch...,3,“Odnoklassniki” Smart TV uchun dastur chiqardi,2016-02-23,https://daryo.uz/2016/02/23/odnoklassniki-smar...
1,2015-yilda o‘zining dastlabki konsert dasturin...,2,Farhod va Shirin dueti keyingi konsertida “hec...,2016-01-17,https://daryo.uz/2016/01/17/farhod-va-shirin-d...
2,Rossiya chempionatining 18-turida hamyurtimiz ...,0,“Krasnodar” “Spartak”ni mag‘lub etgan o‘yinda ...,2015-03-09,https://daryo.uz/2015/03/09/krasnodar-spartakn...
3,Joriy yilda onalik baxtiga musharraf bo‘lgan x...,2,Rayhon egizak bolakaylar ishtirokidagi klipini...,2014-06-11,https://daryo.uz/2014/06/11/rayhon-egizak-bola...
4,Rossiya prezidenti Vladimir Putin topshirig‘i ...,3,Putin topshirig‘iga ko‘ra tuzilgan Rossiyada t...,2016-06-22,https://daryo.uz/2016/06/22/putin-topshirigiga...
...,...,...,...,...,...
2495,Belgiya bosh vaziri Sharl Mishel marafonga sta...,1,"Belgiya malikasi to‘pponchadan o‘q uzib, bosh ...",2017-05-30,https://daryo.uz/2017/05/30/belgiya-malikasi-t...
2496,Estradamizning yorqin yulduzi Shahzodaning 15 ...,2,Rossiyaning “Muz-TV” kanali bosh direktori Sha...,2015-05-11,https://daryo.uz/2015/05/11/rossiyaning-muz-tv...
2497,MDHga a’zo mamlakatlarning Mudofaa vazirlari k...,4,O‘zbekistonda havo sarhadlarining himoya tizim...,2018-04-19,https://daryo.uz/2018/04/19/ozbekistonda-havo-...
2498,"Kecha, 21-sentabr sanasida “Sassuolo”—“Empoli”...",0,"A Seriya. “Inter M” bugun o‘ynaydi, “Milan”, “...",2018-09-22,https://daryo.uz/2018/09/22/seriya-inter-m-bug...


In [32]:
df = pd.DataFrame({
        'text': '[CLS] ' + df['content'].replace(r'\n', ' ', regex= True),
        'label': df['category']
    })

test = pd.DataFrame({
        'text': '[CLS] ' + test['content'].replace(r'\n', ' ', regex= True),
        'label': test['category']
    })

df.head()

,text,label
0,[CLS] AQSh harbiy-dengiz kuchlarining “Uidbi A...,1
1,[CLS] Forbes jurnali dunyoning eng ko‘p haq to...,0
2,[CLS] Xitoyning Shandun shahridagi Sindao pivo...,3
3,[CLS] Olmaota xalqaro aeroporti Facebook’dagi ...,1
4,[CLS] AQShning Los-Anjeles shahrida asli vengr...,2


In [33]:
df = df.sample(frac=1).reset_index(drop=True)


In [ ]:
model = ClassificationModel(
                'bert', 
                'bert-base-multilingual-cased', 
                    num_labels=5, 
                        args=(
                                    {'overwrite_output_dir': True,
                                     'fp16': False,
                                     'num_train_epochs': 2,  
                                     'reprocess_input_data': False,
                                     "learning_rate": 5e-6,                                       
                                     "train_batch_size": 16,
                                     "eval_batch_size": 16,
                                     "weight_decay": 0.2,
                                     "max_seq_length": 256,
                                     "evaluate_during_training_verbose": True,
                                     "evaluate_during_training": True,
                                     "weight_decay": 0,
                                     "do_lower_case": False,
                                     "n_gpu": 4, # can be 1 if you have enough memory
                                     })
                              )
# Train the model
model.train_model(train, eval_df=test)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)
print(result)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

/home/mirzakhalov/anaconda3/envs/transformers/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/mirzakhalov/anaconda3/envs/transformers/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/home/mirzakhalov/anaconda3/envs/transformers/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/home/mirzakhalov/anaconda3/envs/transformers/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:753: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


In [12]:
data = pd.read_json("output_raw.jsonl", lines=True)


In [18]:
result, model_outputs = model.predict(list(data['text_a'].values))

In [19]:
result

array([3, 4, 3, ..., 1, 1, 1])

In [20]:
data.insert(2, "Topic", result, True)

In [22]:
data.to_json("output_raw_with_topics.jsonl", lines=True, orient='records')